参考にしたノートブックはSUJAY KAPADNISのノートブック(https://www.kaggle.com/code/sujaykapadnis/bengali-speech-recognition-for-everyone )、推論部分だけ抜き出してEDAの部分はスキップした。

これはINPUTに学習済みモデルを入れた状態で行う推論から提出のコードのためファインチューニングを行いたいという場合は別のノートブックを作っているのでそちらを参考に

notebook by SUJAY KAPADNIS(https://www.kaggle.com/code/sujaykapadnis/bengali-speech-recognition-for-everyone ) is the reference of this notebook. I used this reference especially inference part, which means I skipped EDA part.

In [1]:
!cp -r ../input/python-packages2 ./

!tar xvfz ./python-packages2/jiwer.tgz
!pip install ./jiwer/jiwer-2.3.0-py3-none-any.whl -f ./ --no-index
!tar xvfz ./python-packages2/normalizer.tgz
!pip install ./normalizer/bnunicodenormalizer-0.0.24.tar.gz -f ./ --no-index
!tar xvfz ./python-packages2/pyctcdecode.tgz
!pip install ./pyctcdecode/attrs-22.1.0-py2.py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/exceptiongroup-1.0.0rc9-py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/hypothesis-6.54.4-py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/pygtrie-2.5.0.tar.gz -f ./ --no-index --no-deps
!pip install ./pyctcdecode/sortedcontainers-2.4.0-py2.py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/pyctcdecode-0.4.0-py2.py3-none-any.whl -f ./ --no-index --no-deps

!tar xvfz ./python-packages2/pypikenlm.tgz
!pip install ./pypikenlm/pypi-kenlm-0.1.20220713.tar.gz -f ./ --no-index --no-deps

jiwer/
jiwer/jiwer-2.3.0-py3-none-any.whl
jiwer/python-Levenshtein-0.12.2.tar.gz
jiwer/setuptools-65.3.0-py3-none-any.whl
Looking in links: ./
Processing ./jiwer/jiwer-2.3.0-py3-none-any.whl
INFO: pip is looking at multiple versions of jiwer to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement python-Levenshtein==0.12.2 (from jiwer) (from versions: none)
ERROR: No matching distribution found for python-Levenshtein==0.12.2
normalizer/
normalizer/bnunicodenormalizer-0.0.24.tar.gz
Looking in links: ./
Processing ./normalizer/bnunicodenormalizer-0.0.24.tar.gz
  Preparing metadata (setup.py) ... done
  Created wheel for bnunicodenormalizer: filename=bnunicodenormalizer-0.0.24-py3-none-any.whl size=17609 sha256=55e37ded2b1a79215e1c89fcc630ba8847bd64064e89d91540a8a5e868e993dc
  Stored in directory: /root/.cache/pip/wheels/78/d7/75/6986dc3616718f950b80e3bd79a796ef618eaef6cd800e7909
Successfully

In [2]:
rm -r python-packages2 jiwer normalizer pyctcdecode pypikenlm

# ライブラリのインストール / Install the libraries

In [3]:
import typing as tp
from pathlib import Path
from functools import partial
from dataclasses import dataclass, field

import pandas as pd
import pyctcdecode
import numpy as np
from tqdm.notebook import tqdm

import librosa

import pyctcdecode
import kenlm
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ProcessorWithLM, Wav2Vec2ForCTC
from bnunicodenormalizer import Normalizer

import cloudpickle as cpkl

import torch
from torch.utils.data import Dataset

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Config

TRAIN, TESTは学習及びテストデータ

SAMPLING_RATE:16000

MODEL_PATHは学習中のoutputを持ってくるか、別のファインチューニングのoutputを持ってくるかの２択

LM_PATHは基本的に固定

===============================

TRAIN:train data

TEST:test data

SAMPLING_RATE:16000

MODEL_PATH:set to the model which is already trained

LM_PATH:basically dont need to change

In [4]:
ROOT = Path.cwd().parent
INPUT = ROOT/"input"
DATA = INPUT/"bengaliai-speech"
TRAIN = DATA/"train_mp3s"
TEST = DATA/"test_mp3s"
SAMPLING_RATE = 16_000

### ファインチューニングした場合はMODEL_PATHを変える。
### If you fine-tune the model, change MODEL_PATH.
MODEL_PATH = INPUT/"wave-2-vec-2-bengali-ai"

### こちらの方は変えなくて良さそう。(他の手法が見当たらない)
### dont need to change this.
LM_PATH = INPUT/"arijitx-full-model/wav2vec2-xls-r-300m-bengali/language_model"

In [5]:
model = Wav2Vec2ForCTC.from_pretrained(MODEL_PATH)
processor = Wav2Vec2Processor.from_pretrained(MODEL_PATH)

vocabularyディクショナリを作成してそれをソート

Wav2Vec2とlanguage modelを組み合わせてdecoderを作成している(decoderは音声データをテキストに変換するタスクを処理するためのもの)。

==============================================================================================================================

make vocabylary dictionaly and then sort it

decoder consists of Wav2Vec2 and language model(decoder processes the conversion from the audio data to the text data)

In [6]:
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k: v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

decoder = pyctcdecode.build_ctcdecoder(
    list(sorted_vocab_dict.keys()), # vocabulary keys
    str(LM_PATH / "5gram.bin"), # path to the language model file
)
processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor, # 音声データの特徴量抽出 extract audio features
    tokenizer=processor.tokenizer, # テキストデータのトークン化 tokenize text data
    decoder=decoder
)

In [7]:
class BengaliSRTestDataset(Dataset):
    # A custom dataset class for handling Bengali speech test data
    
    def __init__(self, audio_paths: list[str], sampling_rate: int):
        # Constructor to initialize the dataset
        
        # Store the list of audio file paths
        self.audio_paths = audio_paths
        
        # Store the sampling rate used for audio processing
        self.sampling_rate = sampling_rate
    
    def __len__(self):
        # Return the total number of samples in the dataset
        return len(self.audio_paths)
    
    def __getitem__(self, index: int):
        # Get a sample from the dataset given an index
        
        # Get the audio file path corresponding to the index
        audio_path = self.audio_paths[index]
        
        # Get the sampling rate from the dataset settings
        sr = self.sampling_rate
        
        # Load the audio file using librosa, specifying the desired sampling rate
        # 'mono=False' indicates to load the audio as a multi-channel signal
        # [0] at the end gets the audio signal (the first element of the returned tuple)
        audio_signal = librosa.load(audio_path, sr=sr, mono=False)[0]
        
        # Return the loaded audio signal as the sample
        return audio_signal

In [8]:
test = pd.read_csv(DATA / "sample_submission.csv", dtype={"id": str})
test_audio_paths = [str(TEST / f"{aid}.mp3") for aid in test["id"].values]
test_dataset = BengaliSRTestDataset(
    test_audio_paths, SAMPLING_RATE
)

collate_func = partial(
    processor_with_lm.feature_extractor,
    return_tensors="pt", sampling_rate=SAMPLING_RATE,
    padding=True,
)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=16, shuffle=False,
    num_workers=2, collate_fn=collate_func, drop_last=False,
    pin_memory=True,
)

# 推論 / Inference

In [9]:
"""
アクセレレータの確認と各種設定

set accelerator and so on
"""

if not torch.cuda.is_available():
    device = torch.device("cpu")
else:
    device = torch.device("cuda")
print(device)
model = model.to(device)

"""
model.eval()とすることでモデルを評価モードに切り替える。

switch to evaluation mode
"""
model = model.eval()

"""
model.half()とすることでモデルのデータ型を半精度浮動小数点数に変換する。
半精度浮動小数点数にすることで演算の高速化、性能向上が期待できる

set to model.half() such that the model is converted to half precision floating point number.
converting to half precision floating point number is expected to speed up the calculation and improve the performance.
"""
model = model.half()

cuda


In [10]:
"""
予測した文を格納するためのリスト

list for storing predicted sentences
"""
pred_sentence_list = []

"""
ファインチューニングは別のノートブックで行っているため購買計算は行わない。
つまり重みを変えるようなコードは書かない。
"""
with torch.no_grad():
    for batch in tqdm(test_loader):
        x = batch["input_values"]
        x = x.to(device, non_blocking=True)
        with torch.cuda.amp.autocast(True):
            y = model(x).logits
        y = y.detach().cpu().numpy()
        
        for l in y:  
            sentence = processor_with_lm.decode(l, beam_width=512).text
            pred_sentence_list.append(sentence)

  0%|          | 0/1 [00:00<?, ?it/s]

# Postprocess と Make submission / Postprocess and make submission

In [11]:
""""
テキストの標準化

normalize text
"""
bnorm = Normalizer()

def postprocess(sentence):
    period_set = set([".", "?", "!", "।"])
    _words = [bnorm(word)['normalized']  for word in sentence.split()]
    sentence = " ".join([word for word in _words if word is not None])
    try:
        if sentence[-1] not in period_set:
            sentence+="।"
    except:
        # print(sentence)
        sentence = "।"
    return sentence

pp_pred_sentence_list = [postprocess(s) for s in tqdm(pred_sentence_list)]

  0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
test["sentence"] = pp_pred_sentence_list

test.to_csv("submission.csv", index=False)

display(test.head())

,id,sentence
0,0f3dac00655e,একটু বয়স হলে একটি বিদেশী।
1,a9395e01ad21,কী কারণে তুমি এতাবৎ কাল পর্যন্ত এ দারুণ দৈব দু...
2,bf36ea8b718d,এ কারণে সরকার নির্ধারিত হারে পরিবহনজনিত ক্ষতি ...
